In [3]:
from collections import deque
from copy import deepcopy
import graphviz as gv

In [4]:
class State:

    def __init__(self, state, parent, empty_pos):
        self.state = state
        self.parent = parent
        self.empty_pos = empty_pos
        self.children = list()

    def toStr(self):
        return str(self.state)

    def printState(self):
        ptr = "{"
        n = len(self.state)
        
        for i in range(n):
            ptr += '{'
            for j in range(n):
                ptr += str(self.state[i][j])
                if j != n - 1:
                    ptr += "|"
            ptr += '}'
            if i != n - 1:
                ptr += '|'

        ptr += '}'
        return ptr

In [25]:
class Puzzle:

    def __init__(self, start, goal):
        self.start = State(state=start, parent=None, empty_pos=self.find_empty_pos(start))
        self.goal = State(state=goal, parent=None, empty_pos=self.find_empty_pos(goal))
        
    def find_empty_pos(self, state):
        n = len(state)
        return [(i, j) for i in range(n) for j in range(n) if state[i][j] == 0][0]

    def getNeighbours(self, curr_state):
        x, y = curr_state.empty_pos
        neighbours = []

        for i, j in zip([1, 0, 0, -1], [0, -1, 1, 0]):
            nx = x + i
            ny = y + j

            if 0 <= nx < 3 and 0 <= ny < 3:
                neighbours.append((nx, ny))

        return neighbours

    def visualize(self, path):
        
        dot = gv.Digraph(comment='Puzzle', format='svg')

        ds = deque()
        ds.append(self.start)

        while ds:
            node = ds.popleft()

            color = 'black'
            if path and node.toStr() == path[-1]:
                path.pop()
                color = 'red'

            uid = str(id(node))
            dot.node(name=uid, label=node.printState(), shape='record', color=color)

            for child in node.children:
                ds.append(child)
                dot.edge(uid, str(id(child)))

        dot.view()

    def solve(self, method='dfs'):

        path = self.traversal(method=method)

        if path:
            self.visualize(path)
        else:
            print('No Solution')
        
    def tracePath(self, curr_state):
        path = []

        while curr_state:
            path.append(curr_state.toStr())
            curr_state = curr_state.parent

        return path

    def traversal(self, method):
        ds = deque()
        ds.append(self.start)
        notAvailable = set()

        while ds:
            curr_state = None

            if method == 'dfs':
                curr_state = ds.pop()
            elif method == 'bfs':
                curr_state = ds.popleft()
            else:
                return None

            if curr_state.toStr() == self.goal.toStr():
                return self.tracePath(curr_state)

            neighbours = self.getNeighbours(curr_state)

            x, y = curr_state.empty_pos
            for (nx, ny) in neighbours:
                curr_state.state[x][y], curr_state.state[nx][ny] = curr_state.state[nx][ny], curr_state.state[x][y]
                
                nextState = State(state=deepcopy(curr_state.state), parent=curr_state, empty_pos=(nx, ny))

                if curr_state.toStr() not in notAvailable:
                    curr_state.children.append(nextState)
                    ds.append(nextState)
                    notAvailable.add(curr_state.toStr())

                curr_state.state[x][y], curr_state.state[nx][ny] = curr_state.state[nx][ny], curr_state.state[x][y]

        return None

In [26]:
start = [
    [1, 8, 2],
    [0, 4, 3],
    [7, 6, 5]
]

end = [
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 0]
]

pz = Puzzle(start, end)
pz.solve(method='bfs')